In [5]:
import numpy as np
import pandas as pd

INPUT_DIR = './input'
TEST_PATH = INPUT_DIR + '/test.csv'
SUBMISSION_PATH = INPUT_DIR + '/sample_submission.csv'

In [6]:
def createAvgStd(df):
    df['avg'] = 0
    for i in range(3, 17):
        df['avg'] += df['measurement_'+str(i)].copy()

    df['avg'] /= 14

    df['std'] = 0
    for i in range(3, 17):
        df['std'] += (df['measurement_'+str(i)].copy() - df['avg'].copy()) ** 2
        df = df.drop(['measurement_'+str(i)].copy(), axis=1)

    df['std'] /= 14 
    df['std'] = np.sqrt(df['std'].copy())

    return df

def dfToNumpy(csv_file):
    df = pd.read_csv(csv_file)
    num_of_rows = len(df['id'])

    df = df.drop(['id', 'product_code', 'attribute_0', 'attribute_1', 'attribute_2', 'attribute_3'], axis=1)   
    
    loading = df['loading'].copy()
    

    for i in range(0, 17):
        df['measurement_'+str(i)] = df['measurement_'+str(i)].copy() / loading.copy()

    df = createAvgStd(df)
    #print("New dataframe columns: {col}, number of columns = {num}".format(col=df.columns.to_list(), num=len(df.columns.to_list())))


    measurement = ['measurement_17', 'measurement_0', 'measurement_1', 'measurement_2']
    no_std = ['std', 'avg']
    need_std = ['loading']
    #need_std = []
    failure = ['failure']

    if 'failure' in df.columns.to_list():
        columns = failure + no_std + need_std + measurement
    else:
        columns = no_std + need_std + measurement

    #print("Data Frame has columns: ", columns)
    df = df[columns]

    #print(df)
    #print("Data Frame has columns: ", df.columns.to_list())

    features = df.to_numpy()
    
    return features, df.columns

### Load Pickle

In [7]:
import pickle

model_name = 'model.pkl'
imputer_name = 'imputer.pkl'
scaler_name = 'scaler.pkl'

model_file = open(model_name, 'rb')
imputer_file = open(imputer_name, 'rb')
scaler_file = open(scaler_name, 'rb')

clf = pickle.load(model_file)
imputer = pickle.load(imputer_file)
scaler = pickle.load(scaler_file)

model_file.close()
imputer_file.close()
scaler_file.close()

print("Done loading")

Done loading


### Inference

In [8]:
features, columns = dfToNumpy(TEST_PATH)
features = imputer.fit_transform(features)

norm_features = features[:, 0:2]
std_features = features[:, 2:]

#norm_features = normalizer.transform(norm_features)
std_features = scaler.transform(std_features)
features = np.concatenate((norm_features, std_features), axis=1)

df = pd.DataFrame(features, columns=list(columns))
print("Data has feature: ", df.columns.to_list())
test_features = df.to_numpy()

y_pred = clf.predict_proba(test_features)[:, 1]
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred.rank()

sample_csv = pd.read_csv(SUBMISSION_PATH)
sample_csv['failure'] = y_pred
sample_csv.to_csv('109550099.csv', index=False)

Data has feature:  ['std', 'avg', 'loading', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2']
